In [1]:
print("hi")

hi


In [8]:
import os
from dotenv import load_dotenv

In [11]:
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [13]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [16]:
from langchain_community.document_loaders import PyPDFLoader

In [26]:
%cd ..

c:\Users\Kings Odenigbo\OneDrive\Desktop\AI\RAG_with FAISS


In [27]:
%pwd

'c:\\Users\\Kings Odenigbo\\OneDrive\\Desktop\\AI\\RAG_with FAISS'

In [28]:
file_path = "data/SDG.pdf"

In [29]:
loader = PyPDFLoader(file_path)
data = loader.load()

In [71]:
len(data)

24

For fewer text use else skip...which will require to covert to Document

In [93]:
gen = ""
for doc in data:
    gen += doc.page_content
    

In [94]:
gen

'IN THE YEAR 2015, LEADERS FROM 193 COUNTRIES OF THE WORLD \nCAME TOGETHER TO FACE THE FUTURE.\nAnd what they saw was daunting. Famines. Drought. Wars. Plagues. Poverty. \nNot just in some faraway place, but in their own cities and towns and villages.\nThey knew things didn’t have to be this way. They knew we had enough \nfood to feed the world, but that it wasn’t getting shared. They knew there \nwere medicines for HIV and other diseases, but they cost a lot. They knew \nthat earthquakes and floods were inevitable, but that the high death \ntolls were not. \nThey also knew that billions of people worldwide shared their hope for a \nbetter future.\nSo leaders from these countries created a plan called the Sustainable \nDevelopment Goals (SDGs). This set of 17 goals imagines a future just 15 years \noff that would be rid of poverty and hunger, and safe from the worst effects of \nclimate change. It’s an ambitious plan. \nBut there’s ample evidence that we can succeed. In the past 15 yea

In [79]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [97]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
texts = text_splitter.split_documents(data)

Not compulsory do run the following steps, its optional

In [99]:
page_text = [t.page_content for t in texts]

In [100]:
type(page_text[0])

str

In [ ]:
from langchain_core.documents import Document
doc = [Document(page_content = t) for t in page_text]

In [103]:
type(doc[0])

langchain_core.documents.base.Document

2nd part 

In [113]:
from langchain_openai import ChatOpenAI

In [114]:

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)


In [122]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [ ]:
from langchain_core.prompts import PromptTemplate


In [124]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=['text'])

In [140]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [141]:
REFINE_PROMPT_QUESTIONS = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)

In [149]:
#from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [152]:
from operator import itemgetter

chain1 = PROMPT_QUESTIONS | llm | StrOutputParser()

chain2 = (
    {"existing_answer": chain1, "text": itemgetter("text")}
    | REFINE_PROMPT_QUESTIONS
    | llm
    | StrOutputParser()
)



In [ ]:
res = chain2.invoke({"existing_answer": "", "text": doc})

In [156]:
print(res)

Based on the provided context, here are refined versions of the original questions to enhance clarity and relevance:

1. **Understanding the Context**: In what year did leaders from 193 countries come together to establish the Sustainable Development Goals (SDGs), and what major global challenges were they aiming to address?

2. **Goals Overview**: How many goals are included in the Sustainable Development Goals (SDGs), and what is the target year set for achieving these goals?

3. **Poverty Reduction**: What commitment was made by the international community in the year 2000 regarding extreme poverty, and what progress was made by 2015?

4. **Hunger and Nutrition**: What significant advancements have been made in reducing global hunger over the past two decades, and what specific strategies are recommended to further eliminate hunger and malnutrition?

5. **Health and Well-Being**: What key statistics are provided regarding child and maternal mortality, and what is the objective for u

In [160]:
from langchain_openai import OpenAIEmbeddings

In [161]:
embeddings = OpenAIEmbeddings()

In [168]:
from langchain_community.vectorstores import FAISS


In [171]:
vectorstore = FAISS.from_documents(documents=doc, embedding=embeddings)

In [181]:
retriever = vectorstore.as_retriever(search_kwargs={"k":3})

In [182]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

In [194]:
answer_prompt = """You are an expert assistant. Use the CONTEXT below to answer the QUESTION "
        "as concisely and accurately as possible.

        CONTEXT: {context}
        
        QUESTION: {question}
        
        ANSWER:
        """

In [195]:
answer_template = PromptTemplate(
    input_variables=["context", "question"],
    template=answer_prompt,
)

In [190]:
llm_chain = (
    {"context": retriever, "question":  RunnablePassthrough()}
    | answer_template
    | llm
    | StrOutputParser()
)

In [191]:
for q in res.split("\n"):
    if q.strip() != "":
        print(f"Question: {q}")
        answer = llm_chain.invoke(q)
        print(f"Answer: {answer}\n\n")

        with open("answers.txt", "a") as f:
            f.write(f"Question: {q}\n")
            f.write(f"Answer: {answer}\n\n")

Question: Based on the provided context, here are refined versions of the original questions to enhance clarity and relevance:
Answer: The provided context emphasizes the interconnection between peace, justice, and sustainable development. It highlights that without peace, a country cannot thrive, and without justice and human rights, peace cannot be achieved. The Sustainable Development Goals aim to reduce violence and promote effective governance, urging countries to strengthen the rule of law and address issues of conflict and insecurity.


Question: 1. **Understanding the Context**: In what year did leaders from 193 countries come together to establish the Sustainable Development Goals (SDGs), and what major global challenges were they aiming to address?
Answer: Leaders from 193 countries came together to establish the Sustainable Development Goals (SDGs) in the year 2015. They aimed to address major global challenges such as famines, drought, wars, plagues, and poverty.


Question